### Introduction

In this assignment, we will build an end to end machine learning pipeline to predict if S&P 500 is likely to move up or down. But first of all, what is S&P500? S&P500 or simply S&P is a "stock market index". A stock market index is designed to replicate the performance of the entire stock market. S&P500, consists of 500 large companies listed on stock exchanges in US; and it is the most commonly followed index globally. You can read more about S&P500 [on this link](https://www.thebalance.com/what-is-the-sandp-500-3305888).

In [182]:
from matplotlib import pyplot as plt # for visualization
from datetime import datetime, timedelta # handle datetime
import pandas as pd
import numpy as np
import re
from sklearn import metrics
import math
import json
from pprint import pprint
# machine learning with sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
import _pickle as cPickle # save ML model
from google.cloud import storage # save the model to GCS

### Data Explanation

In [183]:
# data path
raw_data_path = "gs://mlops-weather-prediction/raw/"
feature_data_path = "gs://mlops-weather-prediction/feature_store/"
model_path = "model_repository/"
tmp_dir = '/tmp/'

# download data
df =  pd.read_csv("raw_data2.csv", error_bad_lines=False)

def dfreplace(df, *args, **kwargs):
    s = pd.Series(df.values.flatten())
    s = s.str.replace(*args, **kwargs)
    return pd.DataFrame(s.values.reshape(df.shape), df.index, df.columns)

weather_df = dfreplace(df, ',', '')

for i in weather_df.columns:
    weather_df[i] = weather_df[i].astype(str)
    weather_df[i][weather_df[i].apply(lambda i: True if re.search('^\s*$', str(i)) else False)]=np.NaN
    
# persist data
weather_df.to_parquet(raw_data_path + 'weather.parquet', compression='GZIP')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
pyarrow or fastparquet is required for parquet support

In [185]:
# let's just explore the data a little bit
weather_df.head()

,"STN,","YYYYMMDD,","DDVEC,","FHVEC,","FG,","FHX,","FHXH,","FHN,","FHNH,","FXX,",...,"VVNH,","VVX,","VVXH,","NG,","UG,","UX,","UXH,","UN,","UNH,",EV24
0,370,20000101,239,21,26,40,3,10,17,70,...,19,38,23,8,99,100,10,98,1,nan
1,370,20000102,194,35,36,50,11,10,1,90,...,4,59,22,8,98,100,2,95,12,nan
2,370,20000103,204,63,63,80,19,40,1,120,...,24,68,10,8,92,97,24,89,18,nan
3,370,20000104,222,53,59,90,17,30,7,170,...,8,75,19,7,93,99,5,77,18,nan
4,370,20000105,193,37,39,50,13,30,1,80,...,8,80,14,3,92,97,8,82,14,nan


In [186]:
# also let's just check for missing data
weather_df[weather_df['TG,']==0].sum()

STN,         0.0
YYYYMMDD,    0.0
DDVEC,       0.0
FHVEC,       0.0
FG,          0.0
FHX,         0.0
FHXH,        0.0
FHN,         0.0
FHNH,        0.0
FXX,         0.0
FXXH,        0.0
TG,          0.0
TN,          0.0
TNH,         0.0
TX,          0.0
TXH,         0.0
T10N,        0.0
T10NH,       0.0
SQ,          0.0
SP,          0.0
Q,           0.0
DR,          0.0
RH,          0.0
RHX,         0.0
RHXH,        0.0
PG,          0.0
PX,          0.0
PXH,         0.0
PN,          0.0
PNH,         0.0
VVN,         0.0
VVNH,        0.0
VVX,         0.0
VVXH,        0.0
NG,          0.0
UG,          0.0
UX,          0.0
UXH,         0.0
UN,          0.0
UNH,         0.0
EV24         0.0
dtype: float64

As we can see from the data above, we have the following information:
* **Open**      : Price for the opening or start of the day
* **High**      : Highest price for that day
* **Low**       : Lowest price for that day
* **Close**     : Closing price
* **Adj Close** : closing price after adjustments for all applicable splits and dividend distributions.
* **Volume**    : Numbers of shares exchanging hands that day (buying & selling)


## Feature Engineering

Our ambition is to predict the closing price of the S&P500 for a given date. However, we only have a few features in our data; and machine learning model typically leverages large feature sets and picks the best features for solving the problem. So in this section, we will create more features from our data.

In [187]:
# holdout data used for evaluation
weather_features_df =  weather_df.drop(weather_df.tail(100).index,inplace=True) # drop last n rows
wather_validate_df = weather_df.tail(100)

TypeError: 'DataFrame' object is not callable

In [188]:
weather_features_df['YYYY'] = weather_features_df['YYYYMMDD,'].str.slice(0,4) #create a variable for years
weather_features_df['MM'] = weather_features_df['YYYYMMDD,'].str.slice(4,6)#create a variable for months
weather_features_df['DD'] = weather_features_df['YYYYMMDD,'].str.slice(6,8)
for i in weather_features_df.columns:
        weather_features_df[i] = weather_features_df[i].astype(float, errors= 'ignore') 
weather_features_df = weather_features_df.drop('YYYYMMDD,', axis=1)
weather_features_df['TG_future'] = weather_features_df['TG,'].shift(periods = -1)

KeyError: 'YYYYMMDD,'

In [189]:
weather_features_df = weather_features_df.drop(columns = ['STN,','EV24', 'NG,', 'TN,', 'TNH,', 'TX,', 'TXH,', 'T10N,', 'T10NH,'])
weather_features_df = weather_features_df.dropna()

KeyError: "['STN,' 'EV24' 'NG,' 'TN,' 'TNH,' 'TX,' 'TXH,' 'T10N,' 'T10NH,'] not found in axis"

In [190]:
weather_features_df

,"DDVEC,","FHVEC,","FG,","FHX,","FHXH,","FHN,","FHNH,","FXX,","FXXH,","TG,",...,"VVXH,","UG,","UX,","UXH,","UN,","UNH,",YYYY,MM,DD,TG_future
0,239.0,21.0,26.0,40.0,3.0,10.0,17.0,70.0,14.0,57.0,...,23.0,99.0,100.0,10.0,98.0,1.0,2000.0,1.0,1.0,70.0
1,194.0,35.0,36.0,50.0,11.0,10.0,1.0,90.0,22.0,70.0,...,22.0,98.0,100.0,2.0,95.0,12.0,2000.0,1.0,2.0,77.0
2,204.0,63.0,63.0,80.0,19.0,40.0,1.0,120.0,12.0,77.0,...,10.0,92.0,97.0,24.0,89.0,18.0,2000.0,1.0,3.0,76.0
3,222.0,53.0,59.0,90.0,17.0,30.0,7.0,170.0,17.0,76.0,...,19.0,93.0,99.0,5.0,77.0,18.0,2000.0,1.0,4.0,44.0
4,193.0,37.0,39.0,50.0,13.0,30.0,1.0,80.0,18.0,44.0,...,14.0,92.0,97.0,8.0,82.0,14.0,2000.0,1.0,5.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,218.0,26.0,53.0,70.0,11.0,30.0,19.0,110.0,11.0,117.0,...,14.0,71.0,88.0,21.0,46.0,15.0,2000.0,4.0,4.0,44.0
95,11.0,71.0,72.0,90.0,3.0,40.0,22.0,140.0,10.0,44.0,...,14.0,69.0,93.0,3.0,52.0,12.0,2000.0,4.0,5.0,43.0
96,38.0,40.0,42.0,60.0,10.0,30.0,1.0,110.0,12.0,43.0,...,1.0,69.0,89.0,23.0,47.0,14.0,2000.0,4.0,6.0,62.0
97,21.0,21.0,21.0,40.0,16.0,0.0,5.0,70.0,16.0,62.0,...,11.0,67.0,94.0,5.0,31.0,16.0,2000.0,4.0,7.0,75.0


### Train  Linear Regression

In [191]:
# write out to parquet
weather_features_df.to_parquet(feature_data_path + 'weather_features_df.parquet', compression='GZIP')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
pyarrow or fastparquet is required for parquet support

In [192]:
# get x and y
x_train, y_train = weather_features_df.drop('TG_future', axis=1), weather_features_df['TG_future']
# split the data for initial testing
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2,random_state=1)
X_train

,"DDVEC,","FHVEC,","FG,","FHX,","FHXH,","FHN,","FHNH,","FXX,","FXXH,","TG,",...,"VVX,","VVXH,","UG,","UX,","UXH,","UN,","UNH,",YYYY,MM,DD
2,204.0,63.0,63.0,80.0,19.0,40.0,1.0,120.0,12.0,77.0,...,68.0,10.0,92.0,97.0,24.0,89.0,18.0,2000.0,1.0,3.0
44,212.0,39.0,42.0,70.0,21.0,20.0,4.0,110.0,24.0,41.0,...,75.0,12.0,84.0,96.0,4.0,49.0,14.0,2000.0,2.0,14.0
59,203.0,71.0,77.0,100.0,12.0,40.0,3.0,180.0,12.0,66.0,...,80.0,13.0,83.0,92.0,23.0,67.0,13.0,2000.0,2.0,29.0
55,287.0,28.0,42.0,60.0,3.0,20.0,19.0,140.0,3.0,48.0,...,80.0,13.0,82.0,97.0,2.0,54.0,16.0,2000.0,2.0,25.0
19,264.0,38.0,38.0,60.0,12.0,20.0,1.0,110.0,14.0,39.0,...,75.0,3.0,90.0,94.0,3.0,81.0,13.0,2000.0,1.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,290.0,39.0,40.0,50.0,1.0,30.0,18.0,100.0,14.0,80.0,...,62.0,8.0,90.0,96.0,24.0,84.0,12.0,2000.0,3.0,16.0
9,185.0,13.0,15.0,30.0,18.0,10.0,1.0,50.0,24.0,-8.0,...,56.0,5.0,99.0,100.0,1.0,95.0,14.0,2000.0,1.0,10.0
72,235.0,35.0,38.0,50.0,10.0,30.0,1.0,80.0,10.0,81.0,...,65.0,1.0,86.0,96.0,18.0,74.0,12.0,2000.0,3.0,13.0
12,183.0,40.0,41.0,60.0,12.0,30.0,7.0,100.0,13.0,23.0,...,70.0,7.0,81.0,94.0,21.0,67.0,14.0,2000.0,1.0,13.0


## Linear regression

In [193]:
model = LinearRegression()
model.fit(X_train, Y_train)

print(model.coef_)

[ 4.21112671e-02  1.23060472e+00 -9.34438343e-01  1.62307246e-01
  4.28747348e-02  3.10880250e-01 -6.23144066e-02 -2.27301438e-01
  1.77735719e-01  6.57609298e-01  1.72098277e-01 -7.29175108e-02
  1.59885916e-03  2.38599690e-01 -3.68946526e-01  5.85384624e-01
  5.90552018e-02  5.52877180e-01 -2.88818789e-01 -4.31829601e-01
 -2.41289403e-01  2.68121906e-01  1.65033433e-01 -1.86889259e-02
 -9.69823443e-01  4.33343911e-01 -4.08017394e-01  1.42248857e+00
  6.78321056e-02 -3.44218448e-01 -7.89492740e-01  4.44089210e-16
  3.13488958e+00  1.62509069e-01]


In [194]:
predictions = model.predict(X_test)
predictions

array([107.18742785,  85.44272538,  33.21718902,  77.58357298,
        39.06198826,  57.62761298,  83.41311965,  96.89567993,
        89.24756766,  54.56502179,  71.24181151,  48.02948283,
        63.94559956,  65.28130349,  24.41229486,  80.00824358,
        67.98780181,  35.47001956,  44.13758061,  48.21366365])

In [195]:
df_test_results = pd.DataFrame({'Prediction':predictions, 'Real': Y_test})
df_test_results

,Prediction,Real
92,107.187428,119.0
83,85.442725,93.0
32,33.217189,57.0
80,77.583573,84.0
78,39.061988,47.0
17,57.627613,42.0
35,83.413120,77.0
81,96.895680,92.0
88,89.247568,64.0
91,54.565022,107.0


In [196]:
MSE = metrics.mean_squared_error(df_test_results['Real'],df_test_results['Prediction'])
RMSE = math.sqrt(MSE)
MAXE = max((df_test_results['Real']- df_test_results['Prediction'])**2)
MAE = metrics.mean_absolute_error(df_test_results['Real'],df_test_results['Prediction'])

pd.Series([MSE,RMSE,MAXE,MAE], index = ['MSE','RMSE','MAXE','MAE'])

MSE      476.072793
RMSE      21.819092
MAXE    2749.426940
MAE       17.678985
dtype: float64

### Random Forest

In [197]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
# Initialize 
model_rf = RandomForestClassifier(random_state = 42, criterion = 'entropy',
                                        max_depth = 3, min_samples_split = 0.1)

model_rf.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', max_depth=3, min_samples_split=0.1,
                       random_state=42)

In [198]:
predictions_2 = model_rf.predict(X_test)
predictions_2

array([44., 37., 44., 75., 75., 75., 75., 63., 44., 44., 66., 78., 55.,
       80., 59., 59., 59., 44., 65., 77.])

In [199]:
df_test_results = pd.DataFrame({'Prediction':predictions_2, 'Real': Y_test})
df_test_results

,Prediction,Real
92,44.0,119.0
83,37.0,93.0
32,44.0,57.0
80,75.0,84.0
78,75.0,47.0
17,75.0,42.0
35,75.0,77.0
81,63.0,92.0
88,44.0,64.0
91,44.0,107.0


### Evaluating The Model

In [200]:
MSE = metrics.mean_squared_error(df_test_results['Real'],df_test_results['Prediction'])
RMSE = math.sqrt(MSE)
MAXE = max((df_test_results['Real']- df_test_results['Prediction'])**2)
MAE = metrics.mean_absolute_error(df_test_results['Real'],df_test_results['Prediction'])

pd.Series([MSE,RMSE,MAXE,MAE], index = ['MSE','RMSE','MAXE','MAE'])

MSE     1260.200000
RMSE      35.499296
MAXE    5625.000000
MAE       26.900000
dtype: float64

### Save The Model

In [201]:
regression_best = model

In [202]:
# save the model into temp
with open('/tmp/model.pickle', 'wb') as f:
    cPickle.dump(regression_best, f, -1)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/model.pickle'